In [ ]:
import requests
import random
import re
from unidecode import unidecode
import pandas as pd
from bs4 import BeautifulSoup
import time
import http
import json


# Function to convert product names to slugs
def product_name_to_slug(product_name):

    product_name = product_name.lower()
    product_name = unidecode(product_name)
    product_name = re.sub(r'\s+', '-', product_name)
    product_name = re.sub(r'[^a-z0-9-]', '', product_name)
    return product_name

def serper_name_to_g2(company_name):
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({
    "q": f"{company_name} alternatives & competitors site:g2.com"
    })
    headers = {
    'X-API-KEY': '46e6377865b21659da0a212efadbadf2129740f5',
    'Content-Type': 'application/json'
    }
    conn.request("POST", "/search", payload, headers)
    res = conn.getresponse()
    data = res.read().decode("utf-8")
    results = json.loads(data)

    organic_results = results['organic']

    g2_url = organic_results[0]['link'] if organic_results else None

    return g2_url

def scrape_g2_alternatives(g2_url):
    apikey = '2fb712f035250fa0feba32543c584318e4894544'

    params = {
        'url': g2_url,
        'apikey': apikey,
        'js_render': 'true',
        'wait_for': '.grid-x',
        'premium_proxy': 'true',
        'proxy_country': 'us',
        'custom_headers': 'true',
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    }
    response = requests.get('https://api.zenrows.com/v1/', params=params, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')
    product_list = soup.find('ul', class_='mb-0 list--chevron')
    products = [li.text for li in product_list.find_all('li')]

    return products

product_competitors = {}
g2_urls = []

def run():
    company_df = pd.read_csv('./data/domains.csv')
    company_names = company_df['domain'].to_list()

    for idx, company_name in enumerate(company_names): 

        print(f"Scraping company {company_name}")

        g2_url = serper_name_to_g2(company_name)
        
        if not g2_url or 'competitors/alternatives' not in g2_url:
            continue

        print(f"Found G2 URL: {g2_url}")
        


        attempts = 0
        success = False

        while attempts < 4 and not success:
            try:
                products = scrape_g2_alternatives(g2_url)
                product_competitors[company_name] = products
                success = True
                print(f'Company: {company_name} competitors: {products}')
            except Exception as e:
                attempts += 1
                wait_time = 2 ** attempts + random.random()  # Exponential backoff with jitter
                print(f"Attempt {attempts} failed: {e}. Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
        
        if not success:
            print(f"Failed to scrape {company_name} after 3 attempts. Skipping to next company.")
            continue
    

run()

Scraping company aligntoday.com
Found G2 URL: https://www.g2.com/products/align/competitors/alternatives
Company: aligntoday.com competitors: ['Rhythm', 'Cascade', 'Perdoo', 'Profit.co', 'Hirebook', 'Peoplebox', 'Mooncamp', 'Quantive']
Scraping company bidtrue.co
Scraping company bonusly.com
Found G2 URL: https://www.g2.com/products/bonusly-bonusly/competitors/alternatives
Company: bonusly.com competitors: ['Kudos', 'Motivosity', 'Achievers', 'Nectar', 'Guusto', 'Awardco', '15Five', 'Vantage Circle']
Scraping company complyup.com
Scraping company boxxstep.com
Found G2 URL: https://www.g2.com/products/boxxstep-brm/competitors/alternatives
Company: boxxstep.com competitors: ['Clari', 'Gainsight Customer Success', 'ChurnZero', 'Totango', 'Aviso', 'Planhat', 'Vitally', 'Catalyst']
Scraping company goeschedule.com
Scraping company cultureally.com
Found G2 URL: https://www.g2.com/products/cultureally/competitors/alternatives
Company: cultureally.com competitors: ['SuccessCOACHING', 'Growth M

In [5]:
import urllib.parse
import re
import requests
import pandas as pd
import maquinillo.settings
import os
from requests.auth import HTTPBasicAuth
import time
import random
import urllib
import http.client
import json
import tldextract

def get_base_domain(url):
    extracted = tldextract.extract(url)
    domain = f"{extracted.domain}.{extracted.suffix}"
    return domain

def sleep_random_time():
    # Generate a random floating-point number between 0 and 2
    random_time = 2 * random.random()

    # Sleep for the random amount of time
    time.sleep(random_time)

def get_domain_from_name(name):
    URL = f"https://company.clearbit.com/v1/domains/find?name={name}"

    CLEARBIT_KEY = os.getenv("CLEARBIT_API_KEY")

    response = requests.get(URL, auth=HTTPBasicAuth(CLEARBIT_KEY, ""))

    if response.status_code == 200:
        response_json = response.json()
        return response_json["domain"]

    elif response.status_code == 404:
        return None

    elif response.status_code == 422:
        print("Weird name:", name)
        return None

    else:
        print("Status code:", response.status_code)
        print("Body: ", response.json())
        raise Exception("Weird scenario")    
    
def serper_name_to_domain(company_name):
    SERPER_KEY = os.getenv("SERPER_API_KEY")
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({
    "q": f"{company_name} company website"
    })
    headers = {
    'X-API-KEY': SERPER_KEY,
    'Content-Type': 'application/json'
    }
    conn.request("POST", "/search", payload, headers)
    res = conn.getresponse()
    data = res.read().decode("utf-8")
    results = json.loads(data)
    website = results['organic'][0]['link']

    if 'linkedin' in website:
        return None

    return get_base_domain(results['organic'][0]['link'])


def name_to_domain(name):
    print(f"Processing {name}")
    
    try:
        clearbit_domain = get_domain_from_name(name)
        if clearbit_domain:
            domain = clearbit_domain
        else:
            domain = serper_name_to_domain(name)
    except Exception as e:
        return name
    return domain

Python-dotenv could not parse statement starting at line 1


In [6]:
print(product_competitors)

for key, values in product_competitors.items():
    product_competitors[key] = [name_to_domain(value) for value in values]

product_competitors_df = pd.DataFrame.from_dict(product_competitors, orient='index')
product_competitors_df.to_csv('./data/annual_only_competitors.csv')


{'aligntoday.com': ['Rhythm', 'Cascade', 'Perdoo', 'Profit.co', 'Hirebook', 'Peoplebox', 'Mooncamp', 'Quantive'], 'bonusly.com': ['Kudos', 'Motivosity', 'Achievers', 'Nectar', 'Guusto', 'Awardco', '15Five', 'Vantage Circle'], 'boxxstep.com': ['Clari', 'Gainsight Customer Success', 'ChurnZero', 'Totango', 'Aviso', 'Planhat', 'Vitally', 'Catalyst'], 'cultureally.com': ['SuccessCOACHING', 'Growth Molecules, Customer Success', 'Hone', 'Wildsparq', 'INTOO', 'FranklinCovey', 'PricewaterhouseCoopers (PwC)', 'LifeLabs Learning'], 'amctechnology.com': ['PandaDoc', 'airSlate SignNow', 'Proposify', 'Jotform Sign', 'Adobe Acrobat Sign', 'GetAccept', 'Qwilr', 'Signeasy'], 'airtable.com': ['ClickUp', 'Asana', 'Teamwork.com', 'Coda', 'Aha!', 'Stackby', 'Hive', 'Smartsheet'], 'shortpoint.com': ['Azure Portal', 'Jahia DXP', 'MangoApps', 'Clinked', 'Oracle WebCenter Content', 'Ingeniux', 'Liferay Digital Experience Platform', 'Budibase'], 'anvilogic.com': ['Cloudflare Application Security and Performanc

In [ ]:
all_competitors = []

for key, values in product_competitors.items():
    all_competitors.extend([value for value in values])

In [9]:
print(all_competitors)

['life-rhythm.net', 'cascadeclean.com', 'perdoo.com', 'profit.co', 'hirebook.com', 'peoplebox.ai', 'mooncamp.com', 'quantive.eu', 'growkudos.com', 'motivosity.com', 'achievers.com', 'nectar.com', 'guusto.com', 'award.co', '15five.com', 'vantagecircle.com', 'clari.com', 'gainsight.com', 'churnzero.com', 'totango.com', 'aviso.ua', 'planhat.com', 'vitally.io', 'catalyst.org', 'successcoaching.co', 'growthmolecules.com', 'honehq.com', 'wildsparq.com', 'intoo.co.jp', 'franklincovey.com', 'pwc.com', 'lifelabslearning.com', 'pandadoc.com', 'signnow.com', 'proposify.com', 'jotform.com', 'adobe.com', 'getaccept.com', 'qwilr.com', 'signeasy.com', 'clickup.com', 'asana.com', 'seetasks.com', 'coda.io', 'aha.io', 'stackby.com', 'hive.com', 'smartsheet.com', 'microsoft.com', 'jahia.com', 'mangoapps.com', 'clinked.com', 'oracle.com', 'ingeniux.com', 'liferay.com', 'budibase.com', 'cloudflare.com', 'microsoft.com', 'mimecast.com', 'crowdstrike.com', 'intezer.com', 'fortinet.com', 'threatlocker.com', '

Quota Attainment: 91.42857142857143
Industry Average Quota Attainment: 40.3
